# Joining and Appending DataFrame

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("joinsAppend").getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "cores")
spark

You are working with 1 cores


In [2]:
# Random DataFrame

valuesP = [('koala', 1, 'yes'), ('caterpillar', 2, 'yes'), ('deer', 3, 'yes'), ('human', 4, 'yes')]
eats_plants = spark.createDataFrame(valuesP, ['name', 'id', 'eats_plants'])

valuesM = [('shark', 5, 'yes'), ('lion', 6, 'yes'), ('tiger', 7, 'yes'), ('human', 4, 'yes')]
eats_meat = spark.createDataFrame(valuesM, ['name', 'id', 'eats_meat'])

print(eats_plants.printSchema())
print(eats_plants.show())

print(eats_meat.printSchema())
print(eats_meat.show())

root
 |-- name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- eats_plants: string (nullable = true)

None
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
root
 |-- name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- eats_meat: string (nullable = true)

None
+-----+---+---------+
| name| id|eats_meat|
+-----+---+---------+
|shark|  5|      yes|
| lion|  6|      yes|
|tiger|  7|      yes|
|human|  4|      yes|
+-----+---+---------+

None


In [3]:
new_df = eats_plants
df_concat = eats_plants.union(new_df)

df_concat.show()
print(("eats_plants df Counts:", eats_plants.count(), len(eats_plants.columns)))
print(("df_cocat Counts:", df_concat.count(), len(df_concat.columns)))
print(eats_plants.show(5))
print(df_concat.show(5))

+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

('eats_plants df Counts:', 4, 3)
('df_cocat Counts:', 8, 3)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      koala|  1|        yes|
+-----------+---+-----------+
only showing top 5 rows

None


In [4]:
inner_join = eats_plants.join(eats_meat, ["name", "id"], "inner")
print("Inner Join Example")
print("inner_join.show()")

Inner Join Example
inner_join.show()


In [5]:
left_join = eats_plants.join(eats_meat, ["name", "id"], how='left')
print("Left Join Example")
print(left_join.show())

Left Join Example
+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+

None


In [6]:
conditional_join = eats_plants.join(eats_meat, ["name","id"], how='left').filter(eats_meat.eats_meat.isNull())
conditional_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



In [7]:
right_join = eats_plants.join(eats_meat, ["name", "id"], how='right')
print(right_join.show())

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|shark|  5|       null|      yes|
|human|  4|        yes|      yes|
|tiger|  7|       null|      yes|
| lion|  6|       null|      yes|
+-----+---+-----------+---------+

None


In [8]:
full_outer_join = eats_plants.join(eats_meat, ["name", "id"], how='full')
print(full_outer_join.show())

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      shark|  5|       null|      yes|
|      human|  4|        yes|      yes|
|      tiger|  7|       null|      yes|
|       lion|  6|       null|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+

None


# Data Set
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses

In [9]:
import os
path = "/user/harishmohan/Datasets/uw-madison-courses/"

fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
result = [file.getPath().getName() for file in list_status]

In [10]:
df_list = []
for filename in result:
    if filename.endswith(".csv"):
        filename_list = filename.split(".")
        df_name = filename_list[0]
        df = spark.read.csv(path+filename, inferSchema= True, header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')

print("Full list of dfs:")
print(df_list)

Full list of dfs:
['course_offerings', 'courses', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subject_memberships', 'subjects', 'teachings']


In [11]:
print('course_offerings',course_offerings.printSchema())
print('courses' ,courses.printSchema())
print('grade_distributions', grade_distributions.printSchema())
print('instructors', instructors.printSchema())
print('rooms', rooms.printSchema())
print('schedules', schedules.printSchema())
print('sections', sections.printSchema())
print('subject_memeberships', subject_memberships.printSchema())
print('subjects', subjects.printSchema())
print('teachings', teachings.printSchema())

root
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)

course_offerings None
root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: integer (nullable = true)

courses None
root
 |-- course_offering_uuid: string (nullable = true)
 |-- section_number: integer (nullable = true)
 |-- a_count: integer (nullable = true)
 |-- ab_count: integer (nullable = true)
 |-- b_count: integer (nullable = true)
 |-- bc_count: integer (nullable = true)
 |-- c_count: integer (nullable = true)
 |-- d_count: integer (nullable = true)
 |-- f_count: integer (nullable = true)
 |-- s_count: integer (nullable = true)
 |-- u_count: integer (nullable = true)
 |-- cr_count: integer (nullable = true)
 |-- n_count: integer (nullable = true)
 |-- p_count: integer (nullable = true)
 |-- i_count: integer (nullable = true)
 |-- nw_count: integer (nullable = true)
 |-- nr_count: in

In [12]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [13]:
instructors.show(5, False)

+-------+------------------+
|id     |name              |
+-------+------------------+
|761703 |JOHN ARCHAMBAULT  |
|3677061|STEPHANIE KANN    |
|788586 |KATHY PREM        |
|1600463|KRISTIN KLARKOWSKI|
|693634 |DAVID BOHNHOFF    |
+-------+------------------+
only showing top 5 rows



In [14]:
teachings.show(3)

+-------------+--------------------+
|instructor_id|        section_uuid|
+-------------+--------------------+
|       761703|45adf63c-48c9-365...|
|       761703|c6280e23-5e43-385...|
|       761703|9395dc21-15d1-3fa...|
+-------------+--------------------+
only showing top 3 rows



In [15]:
step1 = teachings.join(instructors, teachings.instructor_id == instructors.id, how='left').select(['Instructor_id', 'name', 'section_uuid'])
step1.limit(4).toPandas()

,Instructor_id,name,section_uuid
0,761703,JOHN ARCHAMBAULT,45adf63c-48c9-3659-8561-07556d2d4ddf
1,761703,JOHN ARCHAMBAULT,c6280e23-5e43-3859-893e-540d94993529
2,761703,JOHN ARCHAMBAULT,9395dc21-15d1-3fab-8d1f-6f3fe6114c48
3,3677061,STEPHANIE KANN,b99e440b-39db-350a-81eb-b6eb1bd8b0bc


In [17]:
step2 = step1.join(sections, step1.section_uuid == sections.uuid, how = 'left').select(['name', 'course_offering_uuid'])
step2.limit(4).toPandas()

,name,course_offering_uuid
0,THOMAS JAHNS,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,db253216-2e66-3267-86b2-7b9f5fe07223


In [18]:
step3 = step2.withColumnRenamed('name', 'instructor').join(course_offerings, step2.course_offering_uuid == course_offerings.uuid, how ='inner').select(['instructor', 'name', 'course_offering_uuid'])
step3.limit(4).toPandas()

,instructor,name,course_offering_uuid
0,THOMAS JAHNS,Master's Research or Thesis,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,Wrkshp-Industrl Organizatn,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,Workshop - Public Economics,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,Plant Cell Biology,db253216-2e66-3267-86b2-7b9f5fe07223


In [25]:
step1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how='left').select([rooms.uuid, rooms.facility_code, sections.course_offering_uuid,'number'])
step1.limit(4).toPandas()

,uuid,facility_code,course_offering_uuid,number
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,611e5499-724c-3464-82f3-1a5bf29dd96c,307
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,871476fd-340f-303c-a7c6-44ca6ef6b84d,311


In [29]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select([rooms.uuid, rooms.facility_code, 'number', 'term_code', 'name'])
step2.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,307,1182,International Business
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,311,1174,Business Law


In [30]:
step3 = step2.filter(step2.facility_code == "0469")
step3.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,337,1082,Introduction to Philosophy
1,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,352,1082,Introduction to Philosophy
2,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,308,1102,Introduction to Philosophy
3,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,307,1102,Introduction to Philosophy


In [33]:
step1 = subjects.join(subject_memberships, subjects.code == subject_memberships.subject_code, how='inner').select(['name', 'course_offering_uuid']).withColumnRenamed('name', 'subject_name')
step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [37]:
step2 = step1.join(sections, step1.course_offering_uuid == sections.course_offering_uuid, how='left').select(['subject_name', 'room_uuid'])
step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,dd6118e0-7221-3b81-9b29-aad61f0ede54
1,Communication Arts,4e9c55b8-7c02-36c3-b907-3e832ea285e7
2,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
3,Communication Arts,698555af-8ec3-3b85-8e67-bd7fc53aba80


In [38]:
step3 = step2.join(rooms, step2.room_uuid == rooms.uuid, how='left').filter('facility_code IN("0140", "0545", "0469", "0031")').select(['subject_name', 'facility_code', 'room_code'])
step3.limit(4).toPandas()

,subject_name,facility_code,room_code
0,Communication Arts,0140,1070
1,Communication Arts,0545,4028
2,Communication Arts,0545,4008
3,Communication Arts,0469,2241


In [39]:
step3.groupBy('facility_code', 'subject_name').count().orderBy("facility_code").show(10, False)

+-------------+------------------------------------------------+-----+
|facility_code|subject_name                                    |count|
+-------------+------------------------------------------------+-----+
|0031         |Statistics                                      |6    |
|0031         |Interdisciplinary Courses (Engineering)         |1    |
|0031         |Philosophy                                      |1    |
|0031         |Electrical and Computer Engineering             |2    |
|0031         |Interdisciplinary Courses (L&S)                 |1    |
|0031         |History                                         |1    |
|0031         |History of Science                              |1    |
|0031         |Biochemistry                                    |2    |
|0031         |Environmental Studies - Gaylord Nelson Institute|1    |
|0031         |Communication Sciences and Disorders            |5    |
+-------------+------------------------------------------------+-----+
only s

In [40]:
step3.groupBy("subject_name").pivot("facility_code").count().show(10, False)

+---------------------------------+----+----+----+----+
|subject_name                     |0031|0140|0469|0545|
+---------------------------------+----+----+----+----+
|Asian American Studies           |null|6   |245 |24  |
|Religious Studies                |null|32  |282 |43  |
|HEBREW                           |null|7   |4   |null|
|Botany                           |null|4   |24  |2   |
|Urban and Regional Planning      |null|208 |17  |6   |
|Nutritional Sciences             |6   |null|null|28  |
|Kinesiology                      |1549|null|null|null|
|Art Education (Department of Art)|null|null|145 |null|
|Philosophy                       |1   |80  |1296|73  |
|Microbiology                     |null|null|1   |null|
+---------------------------------+----+----+----+----+
only showing top 10 rows



In [43]:
grade_distributions.limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [60]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [62]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step1.limit(4).toPandas()

,course_offering_uuid,sum(f_count)
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4


In [63]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select(['name', 'sum(f_count)']).orderBy("sum(f_count)")
step2.toPandas().tail(5)

,name,sum(f_count)
80166,Calculus&Analytic Geometry,63
80167,Calculus&Analytic Geometry 1,64
80168,Calculus&Analytic Geometry,67
80169,Animal Biology,70
80170,Calc--Functns of Variables,72


In [61]:
print(df_list)
print(" ")
print('course_offerings :', course_offerings.columns)
print('courses :', courses.columns)
print('grade_distributions :', grade_distributions.columns)
print('instructors: ', instructors.columns)
print('rooms :', rooms.columns)
print('schedules :', schedules.columns)
print('sections :', sections.columns)
print('subject_memberships :', subject_memberships.columns)
print('subject :', subjects.columns)
print('teachings :', teachings.columns )

['course_offerings', 'courses', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subject_memberships', 'subjects', 'teachings']
 
course_offerings : ['uuid', 'course_uuid', 'term_code', 'name']
courses : ['uuid', 'name', 'number']
grade_distributions : ['course_offering_uuid', 'section_number', 'a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count', 'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count']
instructors:  ['id', 'name']
rooms : ['uuid', 'facility_code', 'room_code']
schedules : ['uuid', 'start_time', 'end_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
sections : ['uuid', 'course_offering_uuid', 'section_type', 'number', 'room_uuid', 'schedule_uuid']
subject_memberships : ['subject_code', 'course_offering_uuid']
subject : ['code', 'name', 'abbreviation']
teachings : ['instructor_id', 'section_uuid']
